In [ ]:
import pandas as pd
import json
import pickle

In [ ]:
raw_data = pd.read_csv('detailed-data.csv')
# To see all the (unique) devices + data streams:
# raw_data.drop(columns=['from', 'to', 'value']).drop_duplicates()
# device_identifiers = {
#     "Huis": {
#         "Ventilatie_snelheid": ("uuid_a", "FanSpeedPercentage", "FanSpeed"),
#         "Ventilatie_ppm_co2": ("uuid_b", "PartsPerMillion", "CO2"),
#         "Gas_gebruik": ("uuid_c", "CentralMeterGasUsage", "m3"),
#         "Electriciteit_gebruik_hoog": ("uuid_d", "ElectricityUsage", "Wh"),
#         "Electriciteit_gebruik_laag": ("uuid_e", "ElectricityUsage", "Wh"),
#         "Electriciteit_teruglevering_hoog": ("uuid_f", "ElectricityProduction", "Wh"),
#         "Electriciteit_teruglevering_laag": ("uuid_g", "ElectricityProduction", "Wh"),
#     },
#     "Bar": {
#         "Temperatuur_gemeten": ("uuid_a", "ActualTemperatureCelcius", "c"),
#         "Temperatuur_ingesteld": ("uuid_b", "ConfiguredTemperatureCelcius", "c"),
#     },
# }
# kamers = ['Foo', 'Bar']

# _kamers = open("kamers", "w")
# _kamers.write(json.dumps(kamers))
# _kamers.close()

# _device_identifiers = open("device_identifiers", "w")
# _device_identifiers.write(json.dumps(device_identifiers))
# _device_identifiers.close()

In [ ]:
_device_identifiers = open("device_identifiers.json", "r")
device_identifiers = json.loads(_device_identifiers.read())
_device_identifiers.close()

In [ ]:
raw_data['_from'] = raw_data['from'].apply(lambda ts: pd.datetime.fromtimestamp(ts / 1000))
raw_data['_to'] = raw_data['to'].apply(lambda ts: pd.datetime.fromtimestamp(ts / 1000))

In [ ]:
devices = device_identifiers

for section, measurements in devices.items():
    for name, (uuid, c, u) in measurements.items():
        df = raw_data
        df = df[df['uuid'] == uuid]
        df = df[df['category'] == c]
        df = df[df['unit'] == u]
        df = df.set_index('_from')
        #print(df[df.index.duplicated()])
        #print(df[df.index == '2018-10-28 02:00:00']['value'])
        #df.drop_duplicates(subset=None, keep='first', inplace=True)
        #df = df.asfreq('15T')
        df = df.drop(columns=['from', 'to', 'uuid'])
        devices[section][name] = df

In [ ]:
_devices = open("devices.p", "wb")
pickle.dump(devices, _devices)
_devices.close()